In [1]:
import os
import cv2
import pandas as pd
from sklearn.model_selection import KFold
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [2]:
data_path = "datasets/Wildfire-Smoke/train/_annotations.csv"
images_dir = "datasets/Wildfire-Smoke/train/"
output_dir = "result"
os.makedirs(output_dir, exist_ok=True)

In [3]:
data = pd.read_csv(data_path)

In [4]:
def preprocess_images(data, images_dir, output_dir):
    for index, row in data.iterrows():
        filename = row["filename"]
        image_path = os.path.join(images_dir, filename)
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Не удалось загрузить изображение: {filename}")
            continue
        
        xmin, ymin, xmax, ymax = int(row["xmin"]), int(row["ymin"]), int(row["xmax"]), int(row["ymax"])
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, image)

preprocess_images(data, images_dir, output_dir)

In [5]:
def train_model(data_yaml, model_name, epochs, fold, augment, weight_decay, patience):
    model = YOLO(model_name)
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=640,
        augment=augment,
        weight_decay=weight_decay,
        patience=patience,
        verbose=True
    )
    return results.metrics

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
metrics_per_fold = []

for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
    print(f"Fold {fold + 1}")
    
    # Подготовка данных
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]
    
    # Сохранение разбиения
    train_data.to_csv("datasets/Wildfire-Smoke/train_fold.csv", index=False)
    val_data.to_csv("datasets/Wildfire-Smoke/val_fold.csv", index=False)
    
    # Обучение модели
    metrics = train_model(
        data_yaml="datasets/Wildfire-Smoke/data.yaml",
        model_name="yolov8s.pt",
        epochs=15,
        fold=fold,
        augment=True,
        weight_decay=0.0005,
        patience=5
    )
    metrics_per_fold.append(metrics)

Fold 1
New https://pypi.org/project/ultralytics/8.3.49 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.39 🚀 Python-3.11.2 torch-2.2.2 CPU (Intel Core(TM) i5-8257U 1.40GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=datasets/Wildfire-Smoke/data.yaml, epochs=15, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

train: Scanning /Users/baiturtashbaev/Documents/Байтур/Projects/Course Arbeit/INAI_MedFire/datasets/Wildfire-Smoke/train... 272 images, 0 backgrounds, 0 corrupt: 100%|██████████| 272/272 [00:00<00:00, 998.07it/s] 

train: New cache created: /Users/baiturtashbaev/Documents/Байтур/Projects/Course Arbeit/INAI_MedFire/datasets/Wildfire-Smoke/train.cache



val: Scanning /Users/baiturtashbaev/Documents/Байтур/Projects/Course Arbeit/INAI_MedFire/datasets/Wildfire-Smoke/valid... 56 images, 0 backgrounds, 0 corrupt: 100%|██████████| 56/56 [00:00<00:00, 1162.96it/s]

val: New cache created: /Users/baiturtashbaev/Documents/Байтур/Projects/Course Arbeit/INAI_MedFire/datasets/Wildfire-Smoke/valid.cache


...

                   all         56         56     0.0432      0.446     0.0316     0.0119
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



6 epochs completed in 0.678 hours.
Optimizer stripped from /Users/baiturtashbaev/Documents/Байтур/Projects/Course Arbeit/INAI_MedFire/runs/detect/train2/weights/last.pt, 22.5MB
Optimizer stripped from /Users/baiturtashbaev/Documents/Байтур/Projects/Course Arbeit/INAI_MedFire/runs/detect/train2/weights/best.pt, 22.5MB

Validating /Users/baiturtashbaev/Documents/Байтур/Projects/Course Arbeit/INAI_MedFire/runs/detect/train2/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.11.2 torch-2.2.2 CPU (Intel Core(TM) i5-8257U 1.40GHz)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:25<00:00, 12.52s/it]


                   all         56         56      0.527      0.874      0.707      0.426
Speed: 3.3ms preprocess, 423.5ms inference, 0.0ms loss, 11.8ms postprocess per image
Results saved to /Users/baiturtashbaev/Documents/Байтур/Projects/Course Arbeit/INAI_MedFire/runs/detect/train2


AttributeError: 'DetMetrics' object has no attribute 'metrics'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP) of an
    object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (dict of str): A dict of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass plots path and data when

In [7]:
metrics_df = pd.DataFrame(metrics_per_fold)
metrics_df.to_csv("metrics_per_fold.csv", index=False)

plt.figure(figsize=(12, 6))
for metric in ["precision", "recall", "map50"]:
    plt.plot(metrics_df[metric], label=metric)
plt.xlabel("Fold")
plt.ylabel("Value")
plt.title("Metrics across folds")
plt.legend()
plt.savefig("metrics_comparison.png")
plt.show()

KeyError: 'precision'